In [2]:
import time
from datetime import datetime
sc = SparkContext.getOrCreate()
#traces
import pandas as pd
import numpy as np
import re
import sys
import math
#for ordered dict for traces implementation
from collections import OrderedDict

from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

from pyspark.sql import Row

#kmeans
#from pyspark.mllib.clustering import KMeans, KMeansModel
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
#from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [22]:
%run /home/ubuntu/mmscaler_wrapper.py
sc.addPyFile('/home/ubuntu/mmscaler_wrapper.py')

In [1]:
sc

In [23]:
%load_ext autotime

In [24]:
!pwd
!ls /home/ubuntu

/home/ubuntu
autotime.py
autotime.pyc
cluster_centers.csv
cluster_centers_gt10.csv
dave2.csv
dave.csv
derby.log
import_influx.py
Jason-Loading-Visualizing-Endomondo.ipynb
masa-cluster series.ipynb
masa-cluster summary-Copy1.ipynb
masa-cluster summary-Copy2.ipynb
masa-cluster summary.ipynb
metastore_db.old
mmscaler_wrapper.ipynb
mmscaler_wrapper.py
mmscaler_wrapper.pyc
mycert.pem
mykey.key
new_run2.csv
notebooks
pat_alt_metastoredb.ipynb
pypy
README.md
Regression Coefficient Evaluator.ipynb
Regression.ipynb
Regression (Linear) Coefficient Evaluator.ipynb
route_perf_clusters.csv
run_data_with_hr_spd.csv
run_with_hr_users.csv
save-package-versions
sc_test.ipynb
sql_scripts
test.csv
time: 227 ms


In [3]:
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def fix_df(df):
    return df.rdd.toDF(df.schema.names)


def change_column_names(df, old_names, new_names):
    pass
    return df;

##input
##   string: 2013-02-22 21:38:45
##output: 
##   list: [u'2013', u'02', u'22', u'21', u'38', u'45']
def datetime_to_trace_time(timestamp):
    #strip double quotes
    timestamp = timestamp.replace('"', '')
    r_list = re.split('[- :]', timestamp)
    return [int(x) for x in r_list]

In [4]:
df_from_csv('route_perf_clusters.csv')

ValueError: RDD is empty

In [26]:
#get data cell

data_run='new_run2.csv'
data_users='/home/ubuntu/run_with_hr_users.csv'
#df_runs = df_from_csv(data_run)
#print type(df_runs)
#df_runs.count()

time: 1.68 ms


In [73]:
from pyspark.sql import DataFrameReader

url = 'postgresql://172.31.29.1:5432/endomondo'

properties = {'user':'endomondo', 'password': 'End0m0ndo'}

df_runs = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run', properties=properties)

df_users = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run_by_workout', properties=properties)

df_runs

DataFrame[altitude: decimal(20,10), heart_rate: decimal(10,5), latitude: decimal(20,10), longitude: decimal(20,10), speed: decimal(20,10), workoutid: int, time: int, altitude_first: decimal(10,5), altitude_second: decimal(10,5), speed_first: decimal(10,5), elapsed_distance: decimal(15,10), elapsed_time: int, heart_rate_ma_25: decimal(8,5), speed_ma_50: decimal(8,5), speed_ma_100: decimal(8,5)]

time: 31.8 ms


In [79]:
df_runs.schema.names

['altitude',
 'heart_rate',
 'latitude',
 'longitude',
 'speed',
 'workoutid',
 'time',
 'altitude_first',
 'altitude_second',
 'speed_first',
 'elapsed_distance',
 'elapsed_time',
 'heart_rate_ma_25',
 'speed_ma_50',
 'speed_ma_100']

time: 3.16 ms


In [74]:
#clean up df_users
#list of common columns:{'altitude_max',
#  'altitude_min',
#  'calories',
#  'distance',
#  'duration',
#  'elapsed_time',
#  'gender',
#  'geo_distance',
#  'heart_rate_avg',
#  'heart_rate_max',
#  'humidity',
#  'hydration',
#  'id',
#  'series_length',
#  'series_time_delta',
#  'series_time_delta_average',
#  'speed_avg',
#  'speed_max',
#  'start_altitude',
#  'start_latitude',
#  'start_longitude',
#  'start_time',
#  'temperature',
#  'timezone',
#  'userid',
#  'wind_speed',
#  'workoutid'}
df.select([c for c in df.columns if c not in {'GpuName','GPU1_TwoPartHwID'}])
df_users = df_users.select([c for c in df_users.columns if c not in ['elapsed_time']])

time: 73.2 ms


In [61]:
df_users.schema.names

['gender',
 'workoutid',
 'userid',
 'start_time',
 'start_altitude',
 'start_latitude',
 'start_longitude',
 'series_length',
 'series_time_delta',
 'series_time_delta_average',
 'timezone',
 'id',
 'altitude_max',
 'altitude_min',
 'calories',
 'distance',
 'duration',
 'heart_rate_avg',
 'heart_rate_max',
 'hydration',
 'speed_avg',
 'speed_max',
 'humidity',
 'temperature',
 'wind_speed',
 'geo_distance']

time: 4.24 ms


In [57]:
df_runs.schema.names#df_users['workoutid','userid']

['altitude',
 'heart_rate',
 'latitude',
 'longitude',
 'speed',
 'workoutid',
 'time',
 'altitude_first',
 'altitude_second',
 'speed_first',
 'elapsed_distance',
 'elapsed_time',
 'heart_rate_ma_25',
 'speed_ma_50',
 'speed_ma_100']

time: 2.91 ms


In [29]:
#df_users.count()

time: 803 µs


In [31]:
def replacerUDF(value):
    return udf(lambda x: value)

time: 1.22 ms


In [52]:
#class to create summary stats of workouts
class SummaryStat():
    data_points = ''
    new_col_names = []
    #list of tuples. 
    #   key: column name
    #   value: metric (sum/avg/etc)
    summary_conf = ''
    partition_name = ''
    def __init__(self, sql_df, summary_conf = [], \
                partition_name = ''):
        if partition_name:
            self.partition_name = partition_name
        else:
            self.partition_name = 'workoutid'
        self.data_points = sql_df
        if not isinstance(sql_df, DataFrame):
            print 'please enter spark dataframe object'
            return
        #if summary_conf not defined, default to sum for all columns
        if not summary_conf:
            col_names = self.data_points.schema.names
            for i in col_names:
                summary_conf.append((i, 'sum'))
        else:
            if not isinstance(summary_conf, list):
                print 'check summary_conf type'
                return
            for element in summary_conf:
                if not isinstance(element, dict):
                    print 'check summary_conf type 2'
                    return
            self.summary_conf = summary_conf
        
    def describe(self):
        print self.data_points.limit(2)

    def show(self):
        print self.data_points.show()
        
    def get_new_col_names(self):
        return self.new_col_names
        
    def generate_summary(self, debug=False):
        #rewnew new_col_names
        self.new_col_names = []
        #windows function test
        r_df = ''
        r_list_cols = []
        for dic in self.summary_conf:
            a = datetime.now()
            col_name = dic.keys()[0] #should only have 1 key per dictionary
            print col_name
            aggregation_type = dic[col_name]
            w = Window.partitionBy(self.partition_name)
            if aggregation_type == 'avg':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        avg(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())
                self.new_col_names.append(new_col_name)

            if aggregation_type == 'sum':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        sum(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())   
                self.new_col_names.append(new_col_name)

                
            if aggregation_type == 'max':
                #print self.data_points.schema.names
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        max(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())    
                self.new_col_names.append(new_col_name)
                
            if aggregation_type == 'min':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        min(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct()) 
                self.new_col_names.append(new_col_name)
            
            if aggregation_type == 'diff':
                new_col_name = aggregation_type + '_' + col_name
                max_col_name = 'max'+col_name
                min_col_name = 'min'+col_name
                tmp_max = self.data_points.select(col_name, self.partition_name,\
                                                 max(col_name).over(w).alias(max_col_name))
                
                tmp_min = self.data_points.select(col_name, self.partition_name,\
                                        min(col_name).over(w).alias(min_col_name)\
                                            )
                
                min_max_df = tmp_max.join(tmp_min, (tmp_max[self.partition_name]==tmp_min[self.partition_name]),\
                                         'inner').drop(tmp_max[self.partition_name])
                
                min_max_df = min_max_df.withColumn(new_col_name, min_max_df[max_col_name]-min_max_df[min_col_name])
                #print min_max_df[new_col_name, self.partition_name].distinct().show()
                
                r_list_cols.append(min_max_df[[new_col_name, self.partition_name]].distinct())
                self.new_col_names.append(new_col_name)          
                
#                 print min_max_df[[max_col_name, min_col_name, self.partition_name]].show()
#                 #print tmp_max.show(), tmp_min
#                 #tmp_max.select(col_name).subtract(tmp_min.select(col_name)).show()
#                 tmp_max = change_type_col_double(tmp_max, new_col_name)
#                 tmp_min = change_type_col_double(tmp_min, new_col_name)
#                 min_value = tmp_min.select(new_col_name).collect()[0][new_col_name]
#                 max_value = tmp_max.select(new_col_name).collect()[0][new_col_name]
#                 diff = max_value - min_value
#                 workout_id = tmp_max.select(self.partition_name).collect()[0][self.partition_name]
#                 p_name = self.partition_name

#                 #make new dataframe from rdd
#                 k = [(diff, workout_id)]
#                 rdd = sc.parallelize(k)
#                 rdd = rdd.map(lambda x: Row(a=x[0], b=x[1]))
#                 new_df = sqlContext.createDataFrame(rdd)
                
                
#                 #rename column
#                 new_df = new_df.select(col('a').alias(new_col_name), col('b').alias(p_name))
                
#                 r_list_cols.append(new_df)
#                 self.new_col_names.append(new_col_name)   
            b = datetime.now()
            if debug:
                print 'generate stat for ', col_name , ' took ', b-a 
        for df in r_list_cols:
            if not r_df:
                r_df = df
            else:
                r_df = r_df.join(df, (r_df[self.partition_name]==df[self.partition_name]), 'inner')\
                    .drop(df[self.partition_name])
        return r_df
    


time: 155 ms


In [33]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol, s_min = 0, s_max = 0):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.mmModel.setMin(s_min)
        self.mmModel.setMax(s_max)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def denormalize_df(self, df):
        col = self.mmModel.getInputCol()
        
        
    def normalize(self, value):
        pass

time: 25.2 ms


In [34]:
#calculate sse
def compute_sse(a,b):
    total = 0
    if len(a) != len(b):
        print 'bad input'
        return 99999
    else:
        for i in range(len(a)):
            total += math.pow(float(a[i])-float(b[i]),2)
    return math.sqrt(total)

def change_type_col_double(df, col):
    df = df.withColumn(col, df[col].cast("double"))
    return df.na.fill(0)

#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

def normalize_vectorize_df(df, list_cols, scale_dict={}):
    n_df = vectorize_columns(df, list_cols)
    n_df, scalerModels = normalize_df(n_df, list_cols, scale_dict)
    return n_df, scalerModels
#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols, debug=False):
    a = datetime.now()
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    b = datetime.now()
    print 'vectorize done in', b-a
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df_old(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


# MinMaxScaler doesn't have originalMin(only supported in 2.0). made one above cell with wrapper class
# #generate normalized dataframe
# #keep the original column names
# #returns:
# #   (df, {col_name:scalerModel, col_name:scalerModel,....})
# def normalize_df(df, list_cols):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     for col_name in list_cols:
#         scaler = MinMaxScaler(inputCol=col_name, outputCol=tmp_col_name)
#         scalerModel = scaler.fit(df)
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         r_dict[col_name] = [scaler, scalerModel]
#     return df, r_dict

#input: (MLLIB KMEANS)
#   cols_to_cluster: list of column names to cluster
# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
#     vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
#         outputCol='features')
#     df_to_cluster = vecAssembler.transform(df.na.fill(0))
#     df_to_cluster = df_to_cluster.select('features')
#     df_to_cluster = df_to_cluster.rdd\
#         .map(lambda row : Vectors.dense([item for item in row]))
#     clusters = KMeans.train(df_to_cluster, num_clusters,\
#                            maxIterations = 10,\
#                         initializationMode="random")
    
#     return clusters


#create feature column from list of cols
def create_feature_column(df, cols_to_cluster):
    #make dup columns of the ones that will be vectorized
    normalized_names = [i+'_n' for i in cols_to_cluster]
    
    norm_and_woutid = [i+'_n' for i in cols_to_cluster]
    norm_and_woutid.append('workoutid')

    normalized_df = df
    
    for c in range(len(cols_to_cluster)):
        normalized_df = normalized_df.withColumnRenamed(\
                        cols_to_cluster[c], normalized_names[c])
    normalized_df = normalized_df.select(norm_and_woutid)

    
    df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
                'inner').drop(normalized_df['workoutid'])
    #df = df.crossJoin(normalized_df)
    vecAssembler = VectorAssembler(inputCols=normalized_names,\
        outputCol='features')

    df_to_cluster = vecAssembler.transform(df.na.fill(0))

    #normed_df = df_to_cluster.select(normalized_names)
    return df_to_cluster   
    
    
#ML KMEANS
#returns KMeanModel, df_to_cluster
def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=True, bisectingKMean = True,\
                      debug = False):
    a = datetime.now()

#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
#    df_to_cluster = df_to_cluster.select(['workoutid','features'])
    #df_to_cluster = df_to_cluster.rdd\
     #   .map(lambda row : Vectors.dense([item for item in row]))
    df_to_cluster = create_feature_column(df, cols_to_cluster)
    
        
        
    if not bisectingKMean:
        if not init_var:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
                           featuresCol=cols_to_cluster)
        else:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
    else:

        kmeans = BisectingKMeans().setK(num_clusters).setSeed(1).setMinDivisibleClusterSize(10)
    
    
    model = kmeans.fit(df_to_cluster)
    
    b = datetime.now()
    if debug:
        print 'clustering in ', b-a
    return model, df_to_cluster



# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=False):
#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df.copy()
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
# #    df_to_cluster = df_to_cluster.select(['workoutid','features'])
#     #df_to_cluster = df_to_cluster.rdd\
#      #   .map(lambda row : Vectors.dense([item for item in row]))
        
#     print 'clustering times'
#     a = datetime.now()
#     if not init_var:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
#                        featuresCol=cols_to_cluster)
#     else:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
#     b = datetime.now()
#     print b - a
    
#     model = kmeans.fit(df_to_cluster)
    
#     c = datetime.now()
#     print c - b
#     return model, df_to_cluster

    
#returns clusters. show cluster centerse by clusters.clusterCenters.
#output:
#   [0]: KMeanModel
#   [1]: DF that was clustered (normalized)
#   [2]: ScalerModels
#   [3]: error
def cluster_df(df, cols_to_normalize, num_clusters = 2, return_sse = False, cols_to_cluster = [], init_var=True,\
              scale_dict = {}, bisectingKMean = True, debug = False):
    if not cols_to_cluster:
        cols_to_cluster = cols_to_normalize
    df = change_type_cols_double(df, cols_to_normalize)
    df = vectorize_columns(df, cols_to_normalize, debug)
    df, scalerModels = normalize_df(df, cols_to_cluster, scale_dict, debug)
    r_obj = cluster_summary_df(df, cols_to_cluster, num_clusters, init_var, bisectingKMean, \
                              debug)
    sse = 'none'
    if return_sse:
        #calculate SSE for this kmean model
        kmodel = r_obj[0]
        df_to_cluster = r_obj[1]
        centers = kmodel.clusterCenters()
        prediction_df = kmodel.transform(df_to_cluster)

        #set up rdd
        rdd = prediction_df.rdd.map(lambda (a,b):\
                                   (a, centers[b]))
        sse = rdd.map(lambda (a,b): compute_sse(a,b)).sum()
    
    return r_obj[0], r_obj[1], scalerModels, sse

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers



def denormalize_cluster(cluster_centers, list_scaler_models):
    r_centers = []
    for center in cluster_centers:
        r_center = []
        for index in range(len(center)):
            s_model = list_scaler_models[index]
            r_center.append(s_model.denormalize(center[index]))
        r_centers.append(r_center)
    return r_centers

def get_distribution_of_workouts(transformed_df):
    #count of predictions
    return transformed_df.groupBy('prediction').count().orderBy('prediction')

#transform (add prediction column) to the dataframe based on the kmeamodel input
def transform_df_with_kmeamodel(kmeanModel, df, cols_to_cluster, scale_dict={}):
    #need to vectorize and normalize the df
    df_to_transform, scalerModels = normalize_vectorize_df(big_sumstats, cols_to_cluster, scale_dict)
    df_to_transform = create_feature_column(df_to_transform, cols_to_cluster)
    return kmeanModel.transform(df_to_transform), scalerModels



#
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), r[7]]).toDF([\
            'prediction', 'userid', 'workoutid', 'label', 'max_elapsed_time'])

# total_df= fix_df(sumstats.join(prediction_df.select('workoutid','prediction'), \
#                                sumstats['workoutid']==prediction_df['workoutid'],'inner').\
#                             drop(prediction_df['workoutid']))




time: 659 ms


# work on speeding up clustering


In [64]:
sub_df = df_runs.limit(2000)
big_df_to_fit = df_runs.limit(30000)
route_only_df = df_runs.limit(1000)

time: 10.9 ms


In [65]:
sub_df = sub_df.join(df_users,[sub_df['workoutid']==df_users['workoutid']], 'inner').drop(df_users['workoutid'])
big_df_to_fit = big_df_to_fit.join(df_users,[big_df_to_fit['workoutid']==df_users['workoutid']],\
                                   'inner').drop(df_users['workoutid'])


time: 51.1 ms


In [68]:
big_sumstats.limit(2)

DataFrame[max_elapsed_time: int, sum_geo_distance: decimal(25,10), workoutid: int, avg_heart_rate: decimal(14,9), avg_speed: decimal(24,14), diff_altitude: decimal(21,10), gender: string, userid: int, start_time: int, start_altitude: decimal(20,10), start_latitude: decimal(20,10), start_longitude: decimal(20,10), series_length: int, series_time_delta: array<int>, series_time_delta_average: decimal(20,10), timezone: string, id: int, altitude_max: decimal(20,10), altitude_min: decimal(20,10), calories: decimal(20,10), distance: decimal(20,10), duration: decimal(20,10), heart_rate_avg: decimal(20,10), heart_rate_max: decimal(20,10), hydration: decimal(20,10), speed_avg: decimal(20,10), speed_max: decimal(20,10), humidity: int, temperature: int, wind_speed: int, geo_distance: decimal(15,10)]

time: 14.7 ms


In [69]:
#create big sumstats to fit in to kmodel later

#if False:
if True:
    s_conf = [{'elapsed_time': 'max'}, {'geo_distance': 'sum'}, {'heart_rate':'avg'},\
          {'speed':'avg'}, {'altitude': 'diff'}]

    obj = SummaryStat(big_df_to_fit, summary_conf=s_conf)
    #obj = SummaryStat(sub_df, summary_conf=s_conf)

    #obj.describe()
    big_sumstats = obj.generate_summary()
    cols_to_cluster = obj.get_new_col_names()
    print cols_to_cluster
    #clusters = cluster_df(sumstats, cols_to_cluster, 5)
    #clusters.clusterCenters

    #rid nulls
    big_sumstats = big_sumstats.na.drop()

    #attach df_users
    big_sumstats = big_sumstats.join(df_users, [big_sumstats['workoutid']==df_users['workoutid']], 'inner')\
        .drop(df_users['workoutid'])
    #big_sumstats.show()

elapsed_time
geo_distance
heart_rate
speed
altitude
['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
time: 282 ms


In [70]:
#define columns to cluster on (they will be normalized)
#cols_to_cluster = ['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
#create summary stats for workouts

s_conf = [{'elapsed_time': 'max'}, {'geo_distance': 'sum'}, {'heart_rate':'avg'},\
          {'speed':'avg'}, {'altitude': 'diff'}]

obj = SummaryStat(sub_df, summary_conf=s_conf)
#obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
sumstats = obj.generate_summary()
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

#rid nulls
sumstats = sumstats.na.drop()

#attach df_users
sumstats = sumstats.join(df_users, [sumstats['workoutid']==df_users['workoutid']], 'inner')\
    .drop(df_users['workoutid'])
#sumstats.show()

elapsed_time
geo_distance
heart_rate
speed
altitude
['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
time: 272 ms


In [168]:
df = change_type_cols_double(sumstats, cols_to_cluster)
big_sumstats = change_type_cols_double(big_sumstats, cols_to_cluster)



time: 54.9 ms


In [ ]:
###################go to next################

In [169]:
df2 = vectorize_columns(df, cols_to_cluster)

vectorize done in 0:00:00.113043
time: 114 ms


In [194]:
from multiprocessing.pool import ThreadPool
tpool = ThreadPool(processes=6)

time: 2.97 ms


In [190]:
cols_to_cluster = ['max_elapsed_time', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
df3, scalerModels = normalize_df(df2, cols_to_cluster,debug= True)
#df3.show()

------- normalize  max_elapsed_time -------
0:00:11.574995
0:00:00.033739
------- normalize  avg_heart_rate -------
0:00:12.414017
0:00:00.034486
------- normalize  avg_speed -------
0:00:11.597243
0:00:00.035230
------- normalize  diff_altitude -------
0:00:12.340202
0:00:00.038561
time: 48.1 s


In [196]:
cols_to_cluster = ['max_elapsed_time', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
#art = normalize_df2(df2, cols_to_cluster,debug= True)
#df3.show()

time: 7.51 ms


In [188]:
def normalize_col(obj):
    col_name = obj[0]
    print col_name
    df = obj[1]
    tmp_col_name = 'tmp'
    scaler = scaler_wrapper(inputCol=col_name,outputCol=tmp_col_name,\
                           s_max=1)
    scalerModel = scaler.fit(df)
    df = scalerModel.transform(df).drop(col_name)\
        .withColumnRenamed(tmp_col_name, col_name)
    return df, scaler

#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df2(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    arg = zip([d for d in list_cols], [df for i in range(len(list_cols))])
    re = tpool.map(normalize_col, arg)
    print re
#     for col_name in list_cols:
#         if debug:
#             print '------- normalize ', col_name,'-------'
#         a = datetime.now()
#         scale_value = 1
#         if col_name in scale_dict:
#             scale_value = scale_dict[col_name]
#             print scale_value
#         #scaler is the wrapper instance
#         scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
#         scalerModel = scaler.fit(df)
#         b = datetime.now()
#         if debug:
#             print b-a
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         #r_dict[col_name] = scaler
#         r_dict[index] = scaler
#         c =datetime.now()
#         if debug:
#             print c-b
#         index+=1
#     return df, r_dict

time: 22.9 ms


NameError: name 'list_cols' is not defined

time: 10.4 ms


In [171]:
len(df3.collect())

159

time: 4.87 s


In [172]:
r_obj = cluster_summary_df(df3, cols_to_cluster, 5, init_var=True)

KeyboardInterrupt: 

time: 41 s


In [143]:
len(r_obj[1].collect())

36

time: 1.59 s


In [49]:
##output:
#   [0]: KMeanModel
#   [1]: DF that was clustered (normalized)
#   [2]: ScalerModels
#   [3]: error
#r_obj = cluster_summary_df(df3, cols_to_cluster, 10, init_var=True)

clustering times
0:00:00.003325
0:00:10.230032
time: 11.5 s


# clustering based on route info, then in those clusters, cluster based on performance metrics

In [75]:
#sub_df = df_runs.limit(1000)
sub_df = df_runs.limit(100000)

time: 3.8 ms


In [77]:
sub_df.schema.names

['altitude',
 'heart_rate',
 'latitude',
 'longitude',
 'speed',
 'workoutid',
 'time',
 'altitude_first',
 'altitude_second',
 'speed_first',
 'elapsed_distance',
 'elapsed_time',
 'heart_rate_ma_25',
 'speed_ma_50',
 'speed_ma_100']

time: 3.46 ms


In [78]:
df_users.schema.names

['gender',
 'workoutid',
 'userid',
 'start_time',
 'start_altitude',
 'start_latitude',
 'start_longitude',
 'series_length',
 'series_time_delta',
 'series_time_delta_average',
 'timezone',
 'id',
 'altitude_max',
 'altitude_min',
 'calories',
 'distance',
 'duration',
 'heart_rate_avg',
 'heart_rate_max',
 'hydration',
 'speed_avg',
 'speed_max',
 'humidity',
 'temperature',
 'wind_speed',
 'geo_distance']

time: 8.85 ms


In [76]:
#create summary stat as usual

s_conf = [{'elapsed_time': 'max'}, {'geo_distance': 'sum'}, {'heart_rate':'avg'},\
           {'speed':'avg'}, {'altitude': 'diff'}]


#s_conf = [{'geo_distance': 'sum'}, {'altitude': 'diff'}]
obj = SummaryStat(sub_df, summary_conf=s_conf)
#obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
rout_stats = obj.generate_summary(debug=True)
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

#rid nulls
rout_stats = rout_stats.na.drop()

#attach df_users
a = datetime.now()
rout_stats = rout_stats.join(df_users, [rout_stats['workoutid']==df_users['workoutid']], 'inner')\
    .drop(df_users['workoutid'])
b = datetime.now()
print b-a ,'join time'
#rout_stats.show()

elapsed_time
generate stat for  elapsed_time  took  0:00:00.019971
geo_distance


AnalysisException: u"cannot resolve '`geo_distance`' given input columns: [elapsed_time, altitude_first, latitude, speed_ma_50, time, longitude, altitude, heart_rate, speed, workoutid, speed_ma_100, speed_first, altitude_second, elapsed_distance, heart_rate_ma_25];;\n'Project ['geo_distance, workoutid#3657, sum('geo_distance) windowspecdefinition(workoutid#3657, UnspecifiedFrame) AS sum_geo_distance#3793]\n+- GlobalLimit 100000\n   +- LocalLimit 100000\n      +- Relation[altitude#3652,heart_rate#3653,latitude#3654,longitude#3655,speed#3656,workoutid#3657,time#3658,altitude_first#3659,altitude_second#3660,speed_first#3661,elapsed_distance#3662,elapsed_time#3663,heart_rate_ma_25#3664,speed_ma_50#3665,speed_ma_100#3666] JDBCRelation(run) [numPartitions=1]\n"

time: 74.7 ms


In [97]:
#test
route_cols = ['diff_altitude', 'sum_geo_distance']

df = change_type_cols_double(rout_stats, route_cols)
df = vectorize_columns(df, route_cols, debug=True)

vectorize done in 0:00:00.034810
time: 50.5 ms


In [98]:
# def normalize_df2(df, list_cols, scale_dict = {}, debug=False):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     index = 0
#     for col_name in list_cols:
#         if debug:
#             print '------- normalize ', col_name,'-------'
#         a = datetime.now()
#         scale_value = 1
#         if col_name in scale_dict:
#             scale_value = scale_dict[col_name]
#             print scale_value
#         #scaler is the wrapper instance
#         scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
#         scalerModel = scaler.fit(df)
#         b = datetime.now()
#         if debug:
#             print b-a
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         #r_dict[col_name] = scaler
#         r_dict[index] = scaler
#         c =datetime.now()
#         if debug:
#             print c-b
#         index+=1
#     return df, r_dict


# def normalize_df3(df, list_cols, scale_dict = {}, debug=False):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     index = 0
#     list_sw = []
#     for col_name in list_cols:
#         if debug:
#             print '------- normalize ', col_name,'-------'
#         a = datetime.now()
#         scale_value = 1
#         if col_name in scale_dict:
#             scale_value = scale_dict[col_name]
#             print scale_value
#         #scaler is the wrapper instance
#         scaler = mmscaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
#         list_sw.append(scaler)
    
#     sw_rdd2 = sc.parallelize(list_cols)
#     print sw_rdd2.map(lambda r: apply_function(df, r)).collect()
    

        
        
        
# #         #do the fit part in rdd
# #         scalerModel = scaler.fit(df)
# #         b = datetime.now()
# #         if debug:
# #             print b-a
# #         df = scalerModel.transform(df).drop(col_name)\
# #             .withColumnRenamed(tmp_col_name, col_name)
# #         #r_dict[col_name] = scaler
# #         r_dict[index] = scaler
# #         c =datetime.now()
# #         if debug:
# #             print c-b
# #         index+=1
#     return df, r_dict


# def apply_function(df, col_name):
#     scaler = mmscaler_wrapper(inputCol=col_name, outputCol='tmp', s_max = 1.0)
#     scalerModel = scaler.fit(df)
    

time: 9.14 ms


In [99]:

# df, scalerModels = normalize_df3(df, route_cols, debug=True)




time: 19.2 ms


In [101]:
#cluster here, with twice the emphasis on speed
scale_dict = {}
route_cols = ['diff_altitude', 'sum_geo_distance']
#scale_dict['avg_speed']=2
r_obj = cluster_df(rout_stats, route_cols, 5, init_var=True, scale_dict = scale_dict, bisectingKMean=True,\
                  debug=True)
print 'done', route_cols

vectorize done in 0:00:00.041481
------- normalize  diff_altitude -------
0:00:57.329570
0:00:00.033138
------- normalize  sum_geo_distance -------
0:00:41.308670
0:00:00.033781
clustering in  0:09:34.817818
done ['diff_altitude', 'sum_geo_distance']
time: 11min 13s


In [102]:
route_kmodel = r_obj[0]
route_df = r_obj[1]
route_scalerModels = r_obj[2]

time: 2.24 ms


In [ ]:
#######################check if routes make sense

In [157]:
route_prediction_df = route_kmodel.transform(route_df)
distinct_clusters = route_prediction_df.select('prediction').distinct().collect()
route_prediction_df = route_prediction_df.withColumnRenamed('prediction', 'route_prediction')
route_prediction_df = route_prediction_df.withColumnRenamed('features', 'route_features')



time: 20.1 s


In [158]:
route_prediction_df.groupBy('route_prediction').count().show()
route_prediction_df = fix_df(route_prediction_df)

+----------------+-----+
|route_prediction|count|
+----------------+-----+
|               1|  255|
|               3|   98|
|               4|   14|
|               2|  173|
|               0|  801|
+----------------+-----+

time: 39.3 s


In [159]:
route_kmodel.clusterCenters()

[array([ 0.03647154,  0.00633203]),
 array([ 0.03775015,  0.02866293]),
 array([ 0.04249181,  0.08136524]),
 array([ 0.04030573,  0.2396205 ]),
 array([ 0.03707351,  0.56071872])]

time: 17.6 ms


In [160]:
denormalize_cluster(route_kmodel.clusterCenters(),  route_scalerModels)

[[17.370037453183556, 0.0033895760563046181],
 [34.638431372549292, 0.015343436932156869],
 [98.677456647398799, 0.043555301443352588],
 [69.153061224489761, 0.12827029226020409],
 [25.5, 0.30015610317857139]]

time: 8.09 ms


In [162]:
distribution_df = get_distribution_of_workouts(route_kmodel.transform(route_df))
distribution_df.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  801|
|         1|  255|
|         2|  173|
|         3|   98|
|         4|   14|
+----------+-----+

time: 19.8 s


In [ ]:
#use jason's visualization to ensure it looks ok

In [ ]:
#######################end route checker

In [103]:
#distinct_clusters = reg_df.select('cluster').distinct().collect()

#cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

time: 904 µs


In [104]:
#get clusters of routes

route_prediction_df = route_kmodel.transform(route_df)
distinct_clusters = route_prediction_df.select('prediction').distinct().collect()
route_prediction_df = route_prediction_df.withColumnRenamed('prediction', 'route_prediction')
route_prediction_df = route_prediction_df.withColumnRenamed('features', 'route_features')



#get cluster numbers
cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]
cluster_numbers

[1, 3, 4, 2, 0]

time: 19.8 s


In [105]:
route_prediction_df.groupBy('route_prediction').count().show()
route_prediction_df = fix_df(route_prediction_df)

+----------------+-----+
|route_prediction|count|
+----------------+-----+
|               1|  255|
|               3|   98|
|               4|   14|
|               2|  173|
|               0|  801|
+----------------+-----+

time: 39.4 s


In [106]:
# route_prediction_df.schema
# route_prediction_df.where(route_prediction_df['route_prediction']==3)

time: 839 µs


In [107]:
#create list of dataframes for each route_prediction value
list_route_clusters = [route_prediction_df.where(route_prediction_df['route_prediction']==i) \
                      for i in cluster_numbers]

time: 12.4 ms


In [108]:
list_route_clusters[0]

DataFrame[max_elapsed_time: string, avg_heart_rate: double, avg_speed: double, userid: string, diff_altitude: vector, sum_geo_distance: vector, diff_altitude_n: vector, sum_geo_distance_n: vector, workoutid: string, route_features: vector, route_prediction: bigint]

time: 3.19 ms


In [109]:
#got list of dataframes by route cluster. for each of them, create new clusters 
#add a new column 'performance_prediction' for each cluster

performance_cols_to_cluster = ['avg_heart_rate', 'avg_speed', 'max_elapsed_time']

dict_cluster_to_objs = {}

dv_to_dave = ''

for i in range(len(list_route_clusters)):
    print i, 
#     #cluster
#     c_df = list_route_clusters[i]
#     r_obj = cluster_df(c_df, performance_cols_to_cluster, 5, init_var=True, \
#                       scale_dict = scale_dict, bisectingKMean=True,\
#                       debug=False)
#     dict_cluster_to_objs[i] = r_obj

#     #transform the dataset
#     p_kmodel = r_obj[0]
#     p_df_to_transform = r_obj[1]
#     p_scalerModels = r_obj[2]
    
#     p_kmodel.transform(p_df_to_transform)
    c_df = list_route_clusters[i]
    #cluster within the route cluster dataframe
    r_obj = cluster_df(c_df, performance_cols_to_cluster, 3, init_var=True, \
                      scale_dict = scale_dict, bisectingKMean=True,\
                      debug=False)
    dict_cluster_to_objs[i] = r_obj

    #transform the dataset
    p_kmodel = r_obj[0]
    p_df_to_transform = r_obj[1]
    p_scalerModels = r_obj[2]
    print 'transforming'
    temp_df = p_kmodel.transform(p_df_to_transform)
    temp_df = temp_df.withColumnRenamed('features', 'perf_features')
    temp_df = temp_df.withColumnRenamed('prediction', 'perf_prediction')
    #print c_df.schema
    print '\n'
    #transform
    c_df = c_df.drop('perf_features')
    c_df = c_df.drop('perf_prediction')
    list_route_clusters[i] = c_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
                                      temp_df['workoutid']==c_df['workoutid'], 'inner')\
                                        .drop(temp_df['workoutid'])
    #list_route_clusters[i].show()
    #break
    


0 vectorize done in 0:00:00.051490
transforming


1 vectorize done in 0:00:00.044230
transforming


2 vectorize done in 0:00:00.047432
transforming


3 vectorize done in 0:00:00.042685
transforming


4 vectorize done in 0:00:00.050179
transforming


time: 4min 31s


In [110]:
prediction_columns = ['route_predictin', 'perf_prediction']
first = list_route_clusters[0]
sec = list_route_clusters[1]

time: 1.69 ms


In [111]:
frames = [df.toPandas() for df in list_route_clusters]

time: 28.9 s


In [112]:
new_concat_df = sqlContext.createDataFrame(pd.concat(frames))

time: 242 ms


In [113]:
new_concat_df.groupBy(['route_prediction','perf_prediction']).count()\
.orderBy(['route_prediction', 'perf_prediction']).show()

+----------------+---------------+-----+
|route_prediction|perf_prediction|count|
+----------------+---------------+-----+
|               0|              0|  302|
|               0|              1|   75|
|               0|              2|  424|
|               1|              0|  141|
|               1|              1|   88|
|               1|              2|   26|
|               2|              0|   66|
|               2|              1|   16|
|               2|              2|   91|
|               3|              0|    2|
|               3|              1|   51|
|               3|              2|   45|
|               4|              0|    9|
|               4|              1|    5|
+----------------+---------------+-----+

time: 295 ms


In [115]:
#remove small clusters, might be outliers
new_concat_df = new_concat_df.where(
    ((new_concat_df['route_prediction']!= 3) | (new_concat_df['perf_prediction']!= 0))&\
    ((new_concat_df['route_prediction']!= 4) | (new_concat_df['perf_prediction']!= 0))&\
    ((new_concat_df['route_prediction']!= 4) | (new_concat_df['perf_prediction']!= 1))\
)


time: 8.24 ms


In [139]:
#since its got a lot of vectors, get the necessary columns and join with original sumstats
select_columns = ['workoutid', 'route_prediction', 'perf_prediction']

df_to_dave = rout_stats.join(new_concat_df[select_columns], rout_stats['workoutid']==new_concat_df['workoutid'],
                            'inner').drop(new_concat_df['workoutid'])


time: 22.7 ms


In [140]:
df_to_dave.write.csv('route_perf_clusters.csv')


time: 12.7 s


In [ ]:
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), float(r[7])]).toDF([\
          'cluster', 'userid', 'workoutid', 'features', 'label'])

In [141]:
df_to_dave.schema.names

['max_elapsed_time',
 'sum_geo_distance',
 'workoutid',
 'avg_heart_rate',
 'avg_speed',
 'diff_altitude',
 'userid',
 'route_prediction',
 'perf_prediction']

time: 9.25 ms


In [ ]:
#trying to add a new column based on mapping from route/perf_prediction

In [117]:
route_perf_combo = new_concat_df.select(['route_prediction','perf_prediction']).distinct()\
.orderBy(['route_prediction','perf_prediction']).collect()

time: 440 ms


In [122]:
route_perf_to_key = {}
for k in route_perf_combo:
    index = 0
    route_perf_to_key[k] = index
    index += 1

time: 1.88 ms


In [123]:
test = new_concat_df.limit(15)


+----------------+------------------+------------------+--------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+----------------+---------------+--------------------+
|max_elapsed_time|    avg_heart_rate|         avg_speed|  userid|       diff_altitude|    sum_geo_distance|     diff_altitude_n|  sum_geo_distance_n|workoutid|      route_features|route_prediction|perf_prediction|       perf_features|
+----------------+------------------+------------------+--------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+----------------+---------------+--------------------+
|            7322|136.75862068965517| 9.519641379310347| 2974312|[0.03607392489041...|[0.02313031109904...|[0.03607392489041...|[0.02313031109904...|573619302|[0.03607392489041...|               1|              2|[0.28404369667164...|
|             934|156.91256830601094| 9.199239344262292| 435

In [127]:
dd = {}
dd[0]='a'
dd[1]='b'
dd[2]='c'
dd[3]='d'

time: 1.85 ms


In [129]:
dd[0]

'a'

time: 2.3 ms


# Meaningful clustering/distribution
#dataframe creation for dave's regression

In [44]:
#cluster here, with twice the emphasis on speed
scale_dict = {}
#scale_dict['avg_speed']=2
print cols_to_cluster
r_obj = cluster_df(sumstats, cols_to_cluster, 10, init_var=True, scale_dict = scale_dict, bisectingKMean=True)
print 'done', cols_to_cluster

['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
------- max_elapsed_time -------
0:00:10.875038
0:00:00.040240
------- sum_geo_distance -------
0:00:10.802654
0:00:00.042621
------- avg_heart_rate -------
0:00:10.769024
0:00:00.045855
------- avg_speed -------
0:00:10.629080
0:00:00.043338
------- diff_altitude -------
0:00:10.884188
0:00:00.043645
clustering times
-- bisecting --
0:00:00.002955
0:01:34.395306
done ['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
time: 2min 34s


In [45]:
kmodel = r_obj[0]
df = r_obj[1]
scalerModels = r_obj[2]

time: 2.2 ms


In [38]:
cols_to_cluster

['max_elapsed_time',
 'sum_geo_distance',
 'avg_heart_rate',
 'avg_speed',
 'diff_altitude']

time: 2.45 ms


In [53]:
big_prediction_df, big_scalermodels = transform_df_with_kmeamodel(kmodel, big_sumstats, cols_to_cluster)


------- max_elapsed_time -------
0:00:12.582850
0:00:00.042509
------- sum_geo_distance -------
0:00:12.156996
0:00:00.041474
------- avg_heart_rate -------
0:00:12.052505
0:00:00.045938
------- avg_speed -------
0:00:12.163653
0:00:00.049960
------- diff_altitude -------
0:00:12.199930
0:00:00.045493
time: 1min 7s


In [57]:
big_prediction_df.show(2)

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+----------+
| userid|    max_elapsed_time|    sum_geo_distance|      avg_heart_rate|           avg_speed|       diff_altitude|  max_elapsed_time_n|  sum_geo_distance_n|    avg_heart_rate_n|         avg_speed_n|     diff_altitude_n|workoutid|            features|prediction|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+----------+
|4318337|[0.09376563594516...|[0.01381856875191...|[0.45000483045116...|[0.17856427532448...|[0.3137344970142397]|[0.09376563594516...|[0.01381856875191...|[0.45000483045116...|[0.17856427532448...|[0.3137344970142

In [69]:
##join big_prediction_df with the original big_sumstats, to append prediction ID to the original dataframe
big_total_df= fix_df(big_sumstats.join(big_prediction_df.select('workoutid','prediction'), \
                               big_sumstats['workoutid']==big_prediction_df['workoutid'],'inner').\
                               drop(big_prediction_df['workoutid']))




time: 4.29 s


In [99]:
#big_total_df.groupBy('prediction').count().show()
#count < 10 is in cluster 9, 1, 4
df_to_dave = big_total_df[(big_total_df['prediction']!=9) | (big_total_df['prediction']!=1)\
            | (big_total_df['prediction']!=4)]



time: 7.53 ms


In [100]:
#use total_df to find avg_spd/avg_dist for each user in each cluster
# from the predictions, group by userid/clusterid and calculate avg speed and avg distance for each user

w = Window.partitionBy('userid', 'prediction')

new_speed = 'new_avg_speed'
new_dist = 'new_avg_dist'
df_speed = df_to_dave.select('userid', 'prediction',\
                          avg('avg_speed').over(w).alias(new_speed)).distinct()
df_dist = df_to_dave.select('userid', 'prediction',\
                          avg('sum_geo_distance').over(w).alias(new_dist)).distinct()
# total = df_speed.join(df_dist, (df_speed['userid']==df_dist['userid']) & \
#                                (df_speed['prediction']==df_dist['prediction']),\
#                      'inner')
total = df_speed.join(df_dist,(df_speed.prediction==df_dist.prediction) & (df_speed.userid==df_dist.userid),\
                     'inner').drop(df_speed.prediction).drop(df_speed.userid)
total.show()

+------------------+--------+----------+--------------------+
|     new_avg_speed|  userid|prediction|        new_avg_dist|
+------------------+--------+----------+--------------------+
|11.233365517241381| 6873187|         5|0.029164849300000007|
| 6.273719999999999| 7996903|         0|5.826658000000001E-5|
|13.598578723404257| 7178673|         5|0.009407718899999999|
|12.033633262260144| 7404960|         7| 0.10647507740000003|
|11.038550943396226| 4226525|         2|0.010548355799999996|
|11.414899999999998| 2934877|         2|0.003433941699999...|
|  9.77390679082444| 4164701|         2|0.006474015600000001|
|14.191199999999998| 8484809|         0|          8.29646E-5|
|10.781941515837103| 7178673|         6|0.006682765250000002|
| 8.134559999999999| 3324472|         0|0.006826462700000002|
| 9.513321428571428|12406103|         8|0.001905090050000...|
|          8.318265| 7996903|         2|      0.003830854375|
|10.341771428571429| 3191096|         0|0.001653894299999...|
| 5.8409

In [101]:
#join dave df to total
df_to_dave = df_to_dave.join(total,(df_to_dave.prediction==total.prediction) & (df_to_dave.userid==total.userid),\
             'inner').drop(total.prediction).drop(total.userid)


time: 20.7 ms


In [102]:
select_columns = ['prediction', 'userid', 'workoutid', 'sum_geo_distance', 'diff_altitude', \
                 'new_avg_speed', 'new_avg_dist', 'max_elapsed_time']
last_df = df_to_dave.select(select_columns)


time: 17.1 ms


In [112]:
#df_to_dave = vectorizeData(last_df)
df_to_dave = df_to_dave[(df_to_dave['prediction']!=9) & (df_to_dave['prediction']!=1)\
            & (df_to_dave['prediction']!=4)]
print 'writing csv'
df_to_dave.write.csv('dave2.csv', header=True)

writing csv
time: 1min 37s


In [114]:
df_to_dave.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|   44|
|         7|   23|
|         6|   12|
|         5|   47|
|         3|    6|
|         8|   13|
|         2|   93|
+----------+-----+

time: 2min 8s


# centroid creation for Jason's viz

In [62]:
denormed_centers = denormalize_cluster(kmodel.clusterCenters(), scalerModels)
print cols_to_cluster
print denormed_centers

['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
[[208.66666666666669, 0.0070680080708333303, 111.3868282249326, 9.0792079453780286, 4.1083333333333201], [1.0, 6.4958799999999999e-05, 76.0, 50.6646, 3.4000000000000057], [150.15789473684211, 0.0035405021421052625, 134.07917128411185, 10.365930244194519, -6.3263157894736821], [418.85185185185168, 0.0047028594555555562, 145.03624249932204, 10.854931197599797, 6.6222222222222342], [422.03999999999996, 0.014430503424000003, 132.76387073068204, 12.599862303706898, 24.935999999999979], [978.1785714285711, 0.012965297724999998, 156.26848625130913, 11.245726143927582, 6.2642857142857054], [444.58333333333326, 0.010984836750000001, 179.9280348124058, 12.250907544768904, 16.549999999999983], [824.72727272727275, 0.090364553954545457, 159.01416333635436, 12.09374483162426, 15.0], [4655.818181818182, 0.0081534862727272745, 154.95885821495577, 9.5465033909992467, 20.200000000000017], [9980.0, 0.291042739099999

In [63]:
distribution_df = get_distribution_of_workouts(kmodel.transform(df))
#distribution_df.toPandas()
distribution_df.first()

Row(prediction=0, count=24)

time: 4.81 s


In [66]:
df_centroids = pd.DataFrame(denormed_centers, columns=cols_to_cluster).join(distribution_df.toPandas()['count'])#\
#.to_csv('cluster_centers.csv')
df_centroids = df_centroids[df_centroids['count'] > 10]
#df_centroids.to_csv('cluster_centers_gt10.csv')

time: 81.2 ms


In [23]:
norm_cols = [i+'_n' for i in cols_to_cluster]
reg_cols = [i for i in cols_to_cluster]
reg_cols.append('workoutid')
reg_cols.append('features')
reg_cols.append('userid')
n_df = df.select(norm_cols)

time: 27.8 ms


In [24]:
n_df = df.select(reg_cols)
n_df.schema

StructType(List(StructField(max_elapsed_time,VectorUDT,true),StructField(sum_geo_distance,VectorUDT,true),StructField(avg_heart_rate,VectorUDT,true),StructField(avg_speed,VectorUDT,true),StructField(diff_altitude,VectorUDT,true),StructField(workoutid,StringType,true),StructField(features,VectorUDT,true),StructField(userid,StringType,true)))

time: 33.4 ms


In [25]:
prediction_df = kmodel.transform(n_df)


time: 59.8 ms


In [26]:
#join from original dataframe. Adds 'prediction' column to the original sumstats dataframe

total_df= fix_df(sumstats.join(prediction_df.select('workoutid','prediction'), sumstats['workoutid']==prediction_df['workoutid'],'inner').\
    drop(prediction_df['workoutid']))

time: 3.55 s


In [27]:
total_df.schema.names

['max_elapsed_time',
 'sum_geo_distance',
 'workoutid',
 'avg_heart_rate',
 'avg_speed',
 'diff_altitude',
 'userid',
 'prediction']

time: 2.54 ms


In [28]:
#use total_df to find avg_spd/avg_dist for each user in each cluster
# from the predictions, group by userid/clusterid and calculate avg speed and avg distance for each user

w = Window.partitionBy('userid', 'prediction')

new_speed = 'new_avg_speed'
new_dist = 'new_avg_dist'
df_speed = total_df.select('userid', 'prediction',\
                          avg('avg_speed').over(w).alias(new_speed)).distinct()
df_dist = total_df.select('userid', 'prediction',\
                          avg('sum_geo_distance').over(w).alias(new_dist)).distinct()
# total = df_speed.join(df_dist, (df_speed['userid']==df_dist['userid']) & \
#                                (df_speed['prediction']==df_dist['prediction']),\
#                      'inner')
total = df_speed.join(df_dist,(df_speed.prediction==df_dist.prediction) & (df_speed.userid==df_dist.userid),\
                     'inner').drop(df_speed.prediction).drop(df_speed.userid)
total.show()

+------------------+--------+----------+--------------------+
|     new_avg_speed|  userid|prediction|        new_avg_dist|
+------------------+--------+----------+--------------------+
|13.719277130044839|   56331|         1| 0.09948985020000001|
|  9.04588370221328| 7996903|         0| 0.00688404242857143|
|  7.57019076923077| 2174810|         4|        0.0231940093|
|            6.4539| 7996903|         2|2.395346833333333...|
|15.158117647058823|  745030|         0|0.005403330800000001|
| 11.16260689655172| 1812737|         0|        0.0109394422|
|10.341771428571429| 3191096|         2|0.001653894299999...|
| 10.41191020408163| 4057523|         0|0.043932086200000005|
| 7.924976470588236| 3082409|         2|0.013092680100000002|
|           46.2708| 7996903|         3|           2.3708E-6|
|13.366384615384616| 3411439|         0|        0.0059227269|
|10.345069565217395|   80118|         0|0.008080359799999999|
|10.387329230769227| 4362441|         4|0.023242534400000004|
| 7.9107

In [29]:
#join total and total_df
total_df = total_df.join(total,(total_df.prediction==total.prediction) & (total_df.userid==total.userid),\
             'inner').drop(total.prediction).drop(total.userid)

time: 21 ms


In [46]:
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), r[7]]).toDF([\
            'prediction', 'userid', 'workoutid', 'label', 'max_elapsed_time'])

time: 1.99 ms


In [44]:
#select the necessary colmns in order

select_columns = ['prediction', 'userid', 'workoutid', 'sum_geo_distance', 'diff_altitude', \
                 'new_avg_speed', 'new_avg_dist', 'max_elapsed_time']
last_df = total_df.select(select_columns)


time: 17.5 ms


In [50]:
df_to_dave = vectorizeData(last_df)
last_df.write.csv('dave.csv', header=True)

time: 9.4 s


In [51]:
df_from_csv('dave.csv').show()

+----------+-------+---------+--------------------+--------------------+------------------+--------------------+----------------+
|prediction| userid|workoutid|    sum_geo_distance|       diff_altitude|     new_avg_speed|        new_avg_dist|max_elapsed_time|
+----------+-------+---------+--------------------+--------------------+------------------+--------------------+----------------+
|         0|1812737|287717418|        0.0109394422|                 8.6| 11.16260689655172|        0.0109394422|            2607|
|         0|4318337|483910800|0.005144956800000001|   5.399999999999999| 8.933478260869565|0.005144956800000001|             289|
|         4|4362441|586875290|0.023242534400000004|  14.200000000000045|10.387329230769227|0.023242534400000004|            5427|
|         4|2174810|610271083|        0.0231940093|  20.400000000000034|  7.57019076923077|        0.0231940093|            8600|
|         0|4057523|631503257|0.043932086200000005|                15.6| 10.41191020408163

# old testing stuff

In [30]:
cond = [df_speed.userid==total_df.userid, df_speed.prediction==total_df.prediction]


mm = total_df.join(df_dist, cond ,'inner').drop(df_speed.userid).drop(df_speed.prediction)
mm.schema

NameError: name 'df_speed' is not defined

time: 11.4 ms


In [34]:
sub_df_dist = df_dist.limit(3)
sub_df_dist2 = df_dist.limit(3)


time: 25.7 ms


In [48]:
sliced = sqlContext.createDataFrame(total_df.toPandas())
#sliced.show()

time: 1.44 s


In [32]:
ss = total_df.limit(20)
ss.show()

+----------------+--------------------+---------+------------------+------------------+--------------------+-------+----------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|       diff_altitude| userid|prediction|
+----------------+--------------------+---------+------------------+------------------+--------------------+-------+----------+
|            8600|        0.0231940093|610271083|154.58461538461538|  7.57019076923077|  20.400000000000034|2174810|         4|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                 0.0|7996903|         0|
|               9|0.008571358999999997|284139880| 150.0793650793651|           12.4368| -1.3999999999999986|7516129|         0|
|             909| 0.09948985020000001| 62593681| 176.5605381165919|13.719277130044839|-0.20000000000000284|  56331|         1|
|               8|        0.0041011599|442113709|         133.90625| 9.326250000000003|               -2

In [62]:
total_df.write.csv('bad3.csv', header=True)

time: 4.98 s


In [64]:
mydf = df_from_csv('bad3.csv')
mydf.show()

+----------------+--------------------+---------+------------------+------------------+------------------+--------+----------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|     diff_altitude|  userid|prediction|
+----------------+--------------------+---------+------------------+------------------+------------------+--------+----------+
|             702|        0.0059227269|530740947|150.26923076923077|13.366384615384616|              14.6| 3411439|         0|
|              36|         7.082217E-4|379855770|111.33333333333333|            10.548|0.6000000000000005| 7425627|         2|
|               0|          1.40226E-5|305322511|             101.0|           11.8332|               0.0| 7996903|         2|
|              87|0.004678078799999999|492108158| 162.9795918367347| 9.795673469387758|3.6000000000000005|13406764|         0|
|               1|                 0.0|323716970|              89.5|            6.6384|               0.0| 7996

In [65]:
mydf.join(sub_df_dist, (sub_df_dist.prediction==mydf.prediction), 'inner').show()

+----------------+--------------------+---------+------------------+------------------+------------------+--------+----------+--------+----------+--------------------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|     diff_altitude|  userid|prediction|  userid|prediction|        new_avg_dist|
+----------------+--------------------+---------+------------------+------------------+------------------+--------+----------+--------+----------+--------------------+
|             702|        0.0059227269|530740947|150.26923076923077|13.366384615384616|              14.6| 3411439|         0|13406764|         0|0.004678078799999999|
|             702|        0.0059227269|530740947|150.26923076923077|13.366384615384616|              14.6| 3411439|         0|  126898|         0|0.008990285099999997|
|             702|        0.0059227269|530740947|150.26923076923077|13.366384615384616|              14.6| 3411439|         0| 7996903|         0| 0.00688404242

In [66]:
columns = total_df.schema.names
mydf = total_df.rdd.toDF(columns)
mydf.show()

+----------------+--------------------+---------+------------------+------------------+--------------------+-------+----------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|       diff_altitude| userid|prediction|
+----------------+--------------------+---------+------------------+------------------+--------------------+-------+----------+
|            8600|        0.0231940093|610271083|154.58461538461538|  7.57019076923077|  20.400000000000034|2174810|         4|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                 0.0|7996903|         0|
|               9|0.008571358999999997|284139880| 150.0793650793651|           12.4368| -1.3999999999999986|7516129|         0|
|             909| 0.09948985020000001| 62593681| 176.5605381165919|13.719277130044839|-0.20000000000000284|  56331|         1|
|               8|        0.0041011599|442113709|         133.90625| 9.326250000000003|               -2

In [67]:
mydf.join(sub_df_dist, (sub_df_dist.prediction==mydf.prediction), 'inner').show()

+----------------+--------------------+---------+------------------+------------------+-------------------+-------+----------+--------+----------+--------------------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|      diff_altitude| userid|prediction|  userid|prediction|        new_avg_dist|
+----------------+--------------------+---------+------------------+------------------+-------------------+-------+----------+--------+----------+--------------------+
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0|13406764|         0|0.004678078799999999|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0|  126898|         0|0.008990285099999997|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0| 7996903|         0| 0.00688404242

In [35]:
ss.join(sub_df_dist, (sub_df_dist.prediction==ss.prediction), 'inner').show()

+----------------+--------------------+---------+------------------+------------------+-------------------+-------+----------+--------+----------+--------------------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|      diff_altitude| userid|prediction|  userid|prediction|        new_avg_dist|
+----------------+--------------------+---------+------------------+------------------+-------------------+-------+----------+--------+----------+--------------------+
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0|13406764|         0|0.004678078799999999|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0|  126898|         0|0.008990285099999997|
|              26|0.001163178299999...|366407436|             150.2|           9.48456|                0.0|7996903|         0| 7996903|         0| 0.00688404242

In [217]:
sub_df_dist.join(sub_df_dist2, (sub_df_dist.prediction==sub_df_dist2.prediction) &\
                 (sub_df_dist.userid==sub_df_dist2.userid),'inner').show()

+--------+----------+--------------------+--------+----------+--------------------+
|  userid|prediction|        new_avg_dist|  userid|prediction|        new_avg_dist|
+--------+----------+--------------------+--------+----------+--------------------+
| 7996903|         0| 0.00688404242857143| 7996903|         0| 0.00688404242857143|
|  126898|         0|0.008990285099999997|  126898|         0|0.008990285099999997|
|13406764|         0|0.004678078799999999|13406764|         0|0.004678078799999999|
+--------+----------+--------------------+--------+----------+--------------------+

time: 12 s


In [31]:
#use total_df to find avg_spd/avg_dist for each user in each cluster
# from the predictions, group by userid/clusterid and calculate avg speed and avg distance for each user

w = Window.partitionBy('userid', 'prediction')

new_speed = 'new_avg_speed'
new_dist = 'new_avg_dist'
df_speed = total_df.select('userid', 'prediction',\
                          avg('avg_speed').over(w).alias(new_speed)).distinct()
df_dist = total_df.select('userid', 'prediction',\
                          avg('sum_geo_distance').over(w).alias(new_dist)).distinct()
# total = df_speed.join(df_dist, (df_speed['userid']==df_dist['userid']) & \
#                                (df_speed['prediction']==df_dist['prediction']),\
#                      'inner')
total = df_speed.join(df_dist,(df_speed.prediction==df_dist.prediction),\
                     'inner').drop(df_speed.prediction)
total.show()

#             if aggregation_type == 'avg':
#                 new_col_name = aggregation_type + '_' + col_name
#                 tmp = self.data_points.select(col_name, self.partition_name,\
#                                         avg(col_name).over(w).alias(new_col_name)\
#                                             )
#                 r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())
#                 self.new_col_names.append(new_col_name)




Py4JJavaError: An error occurred while calling o717.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 680.0 failed 1 times, most recent failure: Lost task 11.0 in stage 680.0 (TID 9190, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException: 1
	at org.apache.spark.sql.catalyst.expressions.GenericInternalRow.genericGet(rows.scala:200)
	at org.apache.spark.sql.catalyst.expressions.BaseGenericInternalRow$class.getAs(rows.scala:35)
	at org.apache.spark.sql.catalyst.expressions.BaseGenericInternalRow$class.isNullAt(rows.scala:36)
	at org.apache.spark.sql.catalyst.expressions.GenericInternalRow.isNullAt(rows.scala:194)
	at org.apache.spark.sql.catalyst.expressions.JoinedRow.isNullAt(JoinedRow.scala:70)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply5_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apa

time: 3.9 s


In [ ]:
cols_dave = ['workoutid', 'userid', 'prediction', 'max_elapsed_time', ]

In [96]:
prediction_df.withColumn('avg_speed', \
            prediction_df['avg_speed'].stringify).show()

AnalysisException: u"Can't extract value from avg_speed#25883;"

time: 44.1 ms


In [75]:
prediction_df.select(*)

SyntaxError: invalid syntax (<ipython-input-75-d41e7f17bf1f>, line 1)

AnalysisException: u"cannot resolve 'avg(`avg_speed`)' due to data type mismatch: function average requires numeric types, not org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7;;\n'Project [avg_speed#25883, userid#1891, prediction#72750, avg(avg_speed#25883) windowspecdefinition(userid#1891, prediction#72750, UnspecifiedFrame) AS new_avg_speed#72771]\n+- Project [max_elapsed_time#11372, sum_geo_distance#16209, avg_heart_rate#21046, avg_speed#25883, diff_altitude#30720, workoutid#30786, userid#1891, prediction#72750]\n   +- Project [max_elapsed_time#11372, sum_geo_distance#16209, avg_heart_rate#21046, avg_speed#25883, diff_altitude#30720, workoutid#30786, features#34114, userid#1891, UDF(features#34114) AS prediction#72750]\n      +- Project [max_elapsed_time#11372, sum_geo_distance#16209, avg_heart_rate#21046, avg_speed#25883, diff_altitude#30720, workoutid#30786, features#34114, userid#1891]\n         +- Project [userid#1891, max_elapsed_time#11372, sum_geo_distance#16209, avg_heart_rate#

time: 137 ms


In [43]:
#label: time (name of column)
#features: list of feature column names (route info)
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[1:4])]).toDF(['label','features'])

time: 1.76 ms


In [32]:
cols_to_cluster

['max_elapsed_time',
 'sum_geo_distance',
 'avg_heart_rate',
 'avg_speed',
 'diff_altitude']

time: 2.53 ms


In [ ]:
new_col_order = ['workoutid', 'userid', 'prediction', ]

In [50]:
# output of cluster pipe:
# * data frame of
#     * workoutID, userID, clusterID, time it took
#     * column[4] = vector of features (dist/altitude/etc)
#         * also, avg speed/avg dist for this user in this cluster
# change order of columns by selecting columns in the wanted order
df_to_dave = prediction_df
df_to_dave.schema


StructType(List(StructField(max_elapsed_time,VectorUDT,true),StructField(sum_geo_distance,VectorUDT,true),StructField(avg_heart_rate,VectorUDT,true),StructField(avg_speed,VectorUDT,true),StructField(diff_altitude,VectorUDT,true),StructField(workoutid,StringType,true),StructField(prediction,IntegerType,true)))

time: 3.4 ms


In [45]:
vectorizeData(prediction_df).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    4|[0.23312940217895...|
|    0|[0.01169142679038...|
|    0|[0.08615309986666...|
|    1|[1.0,0.8959918159...|
|    0|[0.04122189240164...|
|    2|[4.62881388477555...|
|    0|[0.00434203789765...|
|    0|[0.05171338372363...|
|    0|[0.38855846925378...|
|    2|[0.00190919676347...|
|    2|[0.00711853217766...|
|    4|[0.23361714137951...|
|    2|[1.40945030792698...|
|    2|[0.05197487672968...|
|    0|[0.05953096610452...|
|    0|[0.02044353867164...|
|    0|[0.09036384195902...|
|    0|[0.06872614931326...|
|    0|[0.44157354857490...|
|    2|[0.01151750854681...|
+-----+--------------------+
only showing top 20 rows

time: 1.7 s


In [35]:
vectorizeData(prediction_df, 'max_elapsed_time', ['sum_geo_distance', 'diff_altitude'])

AttributeError: 'DataFrame' object has no attribute 'map'

time: 19 ms


In [31]:
prediction_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+
|    max_elapsed_time|    sum_geo_distance|      avg_heart_rate|           avg_speed|       diff_altitude|workoutid|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+
|               [1.0]|[0.23312940217895...|[0.6698245151075339]|[0.20516416549474...|[0.27319587628866...|610271083|         4|
|[0.00302325581395...|[0.01169142679038...|[0.6246998284734133]|[0.2939477418816262]|[0.14175257731958...|366407436|         0|
|[0.00104651162790...|[0.08615309986666...|[0.6234583027035859]|[0.43086512508000...|[0.13273195876288...|284139880|         0|
|[0.10569767441860...|               [1.0]|[0.8959918159512034]|[0.49034315283435...|[0.1404639175257732]| 62593681|         1|
|[9.30232558139534...|[0.0412218924016432]|[0.4570111492281304]|[0.2866057266883713]|               [0.0

In [179]:
df.select(reg_cols).where(df['workoutid']==323717026).show()

+----------------+--------------------+--------------------+---------+--------------------+---------+
|max_elapsed_time|    sum_geo_distance|      avg_heart_rate|avg_speed|       diff_altitude|workoutid|
+----------------+--------------------+--------------------+---------+--------------------+---------+
|           [0.0]|[2.38295664857680...|[0.33447684391080...|    [2.0]|[0.14175257731958...|323717026|
+----------------+--------------------+--------------------+---------+--------------------+---------+

time: 940 ms


Py4JJavaError: An error occurred while calling o6080.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3076.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3076.0 (TID 13448, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:802)
	at org.

time: 660 ms


In [45]:
def denormalize_cluster(cluster_centers, list_scaler_models):
    r_centers = []
    for center in cluster_centers:
        r_center = []
        for index in range(len(center)):
            s_model = list_scaler_models[index]
            r_center.append(s_model.denormalize(center[index]))
        r_centers.append(r_center)
    return r_centers

<type 'dict'>
time: 1.11 ms


In [52]:
def denormalize_prediction_df(df_workouts, scaler_models):
    r_df = ''
    for k,v in scaler_models.iteritems():
        print k,v.get_input_col_name()
    
    
    for row in df_workouts:
        print row

time: 2.78 ms


In [53]:
denormalize_prediction_df(prediction_df, scalerModels)

0 max_elapsed_time
1 sum_geo_distance
2 avg_heart_rate
3 avg_speed
4 diff_altitude
Column<workoutid>
Column<max_elapsed_time>
Column<sum_geo_distance>
Column<avg_heart_rate>
Column<avg_speed>
Column<diff_altitude>
Column<prediction>
time: 5.7 ms


In [34]:
prediction_df.toPandas()

,workoutid,max_elapsed_time,sum_geo_distance,avg_heart_rate,avg_speed,diff_altitude,features,prediction
0,236736452,[0.000348837209302],[0.000415244368314],[0.102915951973],[0.0],[0.14175257732],"[0.000348837209302, 0.000415244368314, 0.10291...",7
1,284139880,[0.00104651162791],[0.0861530998667],[0.623458302704],[0.43086512508],[0.132731958763],"[0.00104651162791, 0.0861530998667, 0.62345830...",6
2,287717418,[0.303139534884],[0.109955359044],[0.693618028036],[0.371771289412],[0.197164948454],"[0.303139534884, 0.109955359044, 0.69361802803...",9
3,299866442,[0.0110465116279],[0.0812179311131],[0.552390185696],[0.333855987108],[0.261597938144],"[0.0110465116279, 0.0812179311131, 0.552390185...",6
4,305322511,[0.0],[0.000140945030793],[0.118353344768],[0.402871692128],[0.14175257732],"[0.0, 0.000140945030793, 0.118353344768, 0.402...",0
5,323716970,[0.000116279069767],[0.0],[0.0],[0.161950079305],[0.14175257732],"[0.000116279069767, 0.0, 0.0, 0.161950079305, ...",7
6,323717026,[0.0],[2.38295664858e-05],[0.334476843911],[2.0],[0.14175257732],"[0.0, 2.38295664858e-05, 0.334476843911, 2.0, ...",1
7,323717077,[0.302906976744],[0.0115721050709],[0.646655231561],[0.220960755396],[1.0],"[0.302906976744, 0.0115721050709, 0.6466552315...",5
8,323717705,[0.00104651162791],[0.0204435386716],[0.600343053173],[0.233116286835],[0.16881443299],"[0.00104651162791, 0.0204435386716, 0.60034305...",6
9,323717755,[0.115813953488],[0.388558469254],[0.599594134274],[0.273654168661],[0.137886597938],"[0.115813953488, 0.388558469254, 0.59959413427...",4


time: 1.47 s


In [52]:
prediction_df.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[3.48837209302325...|         7|
|[0.00104651162790...|         6|
|[0.30313953488372...|         9|
|[0.01104651162790...|         6|
|[0.0,1.4094503079...|         0|
|[1.16279069767441...|         7|
|[0.0,2.3829566485...|         1|
|[0.30290697674418...|         5|
|[0.00104651162790...|         6|
|[0.11581395348837...|         4|
|[0.00302325581395...|         6|
|[9.30232558139534...|         0|
|[0.00418604651162...|         0|
|[0.08837209302325...|         6|
|[6.97674418604651...|         7|
|[9.30232558139534...|         0|
|[0.00104651162790...|         0|
|[0.09116279069767...|         6|
|[0.03360465116279...|         6|
|[0.01011627906976...|         6|
+--------------------+----------+
only showing top 20 rows

time: 1.44 s


In [41]:
denormalize_cluster(prediction_df, scalerModels)

TypeError: object of type 'Column' has no len()

time: 13.2 ms


In [40]:
prediction_df.collect()

[Row(features=DenseVector([0.0003, 0.0004, 0.1029, 0.0, 0.1418]), prediction=7),
 Row(features=DenseVector([0.001, 0.0862, 0.6235, 0.4309, 0.1327]), prediction=6),
 Row(features=DenseVector([0.3031, 0.11, 0.6936, 0.3718, 0.1972]), prediction=9),
 Row(features=DenseVector([0.011, 0.0812, 0.5524, 0.3339, 0.2616]), prediction=6),
 Row(features=DenseVector([0.0, 0.0001, 0.1184, 0.4029, 0.1418]), prediction=0),
 Row(features=DenseVector([0.0001, 0.0, 0.0, 0.162, 0.1418]), prediction=7),
 Row(features=DenseVector([0.0, 0.0, 0.3345, 2.0, 0.1418]), prediction=1),
 Row(features=DenseVector([0.3029, 0.0116, 0.6467, 0.221, 1.0]), prediction=5),
 Row(features=DenseVector([0.001, 0.0204, 0.6003, 0.2331, 0.1688]), prediction=6),
 Row(features=DenseVector([0.1158, 0.3886, 0.5996, 0.2737, 0.1379]), prediction=4),
 Row(features=DenseVector([0.003, 0.0117, 0.6247, 0.2939, 0.1418]), prediction=6),
 Row(features=DenseVector([0.0009, 0.0115, 0.4671, 0.1952, 0.3866]), prediction=0),
 Row(features=DenseVecto

time: 1.37 s


In [ ]:
denormed_centers

In [58]:
for k,v in scalerModels.iteritems():
    print k, v.get_input_col_name()

0 max_elapsed_time
1 sum_geo_distance
2 avg_heart_rate
3 avg_speed
4 diff_altitude
time: 1.95 ms


In [32]:
for i in range(5,130,30):
    print i
    r_obj = cluster_summary_df(df3, cols_to_cluster, 5, init_var=True)

5
clustering times
0:00:00.003365
0:03:33.976420
35
clustering times
0:00:00.003368
0:03:32.754669
65
clustering times
0:00:00.003404
0:03:32.487441
95
clustering times
0:00:00.003227
0:03:32.888310
125
clustering times
0:00:00.003173
0:03:31.838358
time: 18min 55s


In [40]:
cluster_to_sse_dict = {}
for i in range(5, 70, 5):
    print i, 
    r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=i,\
                   return_sse = True)
    cluster_to_sse_dict[i] = r_obj[3]

5 ------- max_elapsed_time -------
0:00:03.771995
0:00:00.031710
------- sum_geo_distance -------
0:00:03.697294
0:00:00.032369
------- avg_heart_rate -------
0:00:03.684409
0:00:00.032021
------- avg_speed -------
0:00:03.766936
0:00:00.032681
------- diff_altitude -------
0:00:03.927165
0:00:00.034656
clustering times
0:00:00.002907
0:00:09.387528
10 ------- max_elapsed_time -------
0:00:03.722428
0:00:00.031690
------- sum_geo_distance -------
0:00:03.874099
0:00:00.033990
------- avg_heart_rate -------
0:00:03.784784
0:00:00.032303
------- avg_speed -------
0:00:03.747476
0:00:00.032486
------- diff_altitude -------
0:00:03.785116
0:00:00.033960
clustering times
0:00:00.003148
0:00:10.384973
15 ------- max_elapsed_time -------
0:00:03.765619
0:00:00.032495
------- sum_geo_distance -------
0:00:03.701715
0:00:00.032304
------- avg_heart_rate -------
0:00:03.771162
0:00:00.032318
------- avg_speed -------
0:00:03.850454
0:00:00.034169
------- diff_altitude -------
0:00:03.657665
0:00

KeyboardInterrupt: 

time: 1min 57s


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


cluster_to_sse_dict[i] = r_obj[3]

for k in sorted(cluster_to_sse_dict):
    x = k
    y = cluster_to_sse_dict5[k]
    plt.scatter(x,y)
plt.legend()
plt.show()


# Cluster section


In [33]:
df_runs.schema

StructType(List(StructField(altitude,StringType,true),StructField(heart_rate,StringType,true),StructField(latitude,StringType,true),StructField(longitude,StringType,true),StructField(speed,StringType,true),StructField(workoutid,StringType,true),StructField(time,StringType,true),StructField(altitude_first,StringType,true),StructField(altitude_second,StringType,true),StructField(elapsed_time,StringType,true),StructField(speed_first,StringType,true),StructField(geo_distance,StringType,true)))

time: 2.31 ms


In [34]:
#define columns to cluster on (they will be normalized)
#cols_to_cluster = ['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
#create summary stats for workouts

s_conf = [{'elapsed_time': 'max'}, {'geo_distance': 'sum'}, {'heart_rate':'avg'},\
          {'speed':'avg'}, {'altitude': 'diff'}]

obj = SummaryStat(df_runs, summary_conf=s_conf)
#obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
sumstats = obj.generate_summary()
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
sumstats.show()
sumstats.count()
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

#rid nulls
sumstats = sumstats.na.drop()

['max_elapsed_time', 'sum_geo_distance', 'avg_heart_rate', 'avg_speed', 'diff_altitude']
+----------------+--------------------+---------+------------------+------------------+------------------+
|max_elapsed_time|    sum_geo_distance|workoutid|    avg_heart_rate|         avg_speed|     diff_altitude|
+----------------+--------------------+---------+------------------+------------------+------------------+
|             908|0.014136676100000002|202885174|              null|11.894492307692307|              31.4|
|            2452|         5.398919E-4|278888647|              null|10.578000000000001|10.199999999999989|
|               8|0.028470258399999993|280919215|              null|10.904192307692306|20.599999999999994|
|             320|         8.080148E-4|315716952|              null|           11.5902|0.5999999999999659|
|             987|        0.0863023672|391330335| 161.9485294117647|15.580270588235297| 98.80000000000001|
|             998|  0.3070946213000001|408722698|      

In [35]:
# r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=2,\
#                    return_sse = True) 
# kmodel = r_obj[0]
# new_df = r_obj[1]
# scaler_model_dict = r_obj[2]
# error = r_obj[3]
# print error

time: 1.28 ms


In [36]:
cluster_to_sse_dict5 = {}
for i in range(5, 70, 5):
    print i, 
    r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=i,\
                   return_sse = True)
    cluster_to_sse_dict5[i] = r_obj[3]

5 ------- max_elapsed_time -------
0:00:31.305642
0:00:00.036586
------- sum_geo_distance -------
0:00:29.135043
0:00:00.036809
------- avg_heart_rate -------
0:00:27.274570
0:00:00.036803
------- avg_speed -------
0:00:26.355532
0:00:00.035894
------- diff_altitude -------
0:00:26.458103
0:00:00.036998
clustering times
0:00:00.003163


KeyboardInterrupt: 

time: 3min 19s


In [ ]:
for k in sorted(cluster_to_sse_dict5):
    print k, cluster_to_sse_dict5[k]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline




In [ ]:
for k in sorted(cluster_to_sse_dict5):
    x = k
    y = cluster_to_sse_dict5[k]
    plt.scatter(x,y)
plt.legend()
plt.show()

In [ ]:
#try with a K
r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=5,\
                   return_sse = True)


In [ ]:
kmodel = r_obj[0]
df = r_obj[1]
scale_model = r_obj[2]
sse = r_obj[3]
transformed_df = kmodel.transform(df)

In [ ]:
print s_conf
denormed_centers = denormalize_cluster(kmodel.clusterCenters(), scale_model)

#df = pd.DataFrame.from_records(sales, columns=labels)
pd.DataFrame.from_records(denormed_centers, columns = cols_to_cluster)
#write out to csv to visualize better

#kmodel.clusterCenters()

In [ ]:
#count of predictions
transformed_df.groupBy('prediction').count().orderBy('prediction').show()

In [148]:
#thread test
from pyspark import SparkContext
import numpy.random as rand
import timeit
from multiprocessing.pool import ThreadPool
tpool = ThreadPool(processes=4)
from pyspark.mllib.clustering import KMeans
import numpy as np
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 166 ms


In [149]:
def error(point, clusters):
    center = clusters.centers[clusters.predict(point)]
    return np.linalg.norm(point - center)

def calc_wssse(i):
    clusters = KMeans.train(c_points, i, maxIterations=20,
        runs=20, initializationMode="random")
    WSSSE = c_points\
        .map(lambda point: error(point, clusters))\
        .reduce(lambda x, y: x + y)
    return (i, WSSSE)

time: 4.41 ms


In [153]:
c_points = sc.parallelize([rand.randint(10, size=12) for i in range(10000)])

time: 60.4 ms


In [154]:
for i in range(1,30):
    calc_wssse(i)

time: 33.4 s


In [155]:
wssse_points = tpool.map(calc_wssse, range(1,30))


time: 19.9 s


 # custering section 2

In [ ]:
#define columns to cluster on (they will be normalized)
#cols_to_cluster = ['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
#create summary stats for workouts

s_conf = [{'elapsed_time': 'max'}, {'geo_distance': 'sum'}, \
          {'heart_rate':'avg'}, {'speed':'avg'}]

obj = SummaryStat(df_runs, summary_conf=s_conf)
#obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
sumstats = obj.generate_summary()
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
sumstats.show()
sumstats.count()
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

#rid nulls
sumstats = sumstats.na.drop()